In [ ]:
import pandas as pd
import psycopg2 as psys
import os
import sys
import plotly.express as pt
from warnings import filterwarnings
from datetime import datetime
from meteostat import Hourly
from dotenv import load_dotenv

start = datetime(2023,7,1,00,00,00,00)
end = datetime(2023,8,31,23,59,59,59)

#10675 is Bamberg
data = Hourly(10675,start,end)
weather_data = data.fetch().reset_index()
weather_data = weather_data[['time', 'temp', 'prcp', 'coco']]

print(weather_data.columns)
filterwarnings("ignore", category=UserWarning, message='.*pandas only supports SQLAlchemy connectable.*')




load_dotenv() # take environment variables from .env.
%matplotlib inline

conn = None
def connect():
    try:
        print('Connecting…')
        conn = psys.connect(
            host=os.environ['DB_HOST'],
            database=os.environ['DB_NAME'],
            user=os.environ['DB_USERNAME'],
            password=os.environ['DB_PASSWORD'])
    except (Exception, psys.DatabaseError) as error:
       print(error)
       sys.exit(1)
    print('All good, Connection successful!')
    return conn

conn = connect()
cur = conn.cursor()
query_one = f"""SELECT to_timestamp(floor((extract('epoch' from epocutc)/600))*600)
AT TIME ZONE 'UTC' as time, COUNT(DISTINCT mac_address) as people_count
FROM testschemadb.flowtrack_raw
WHERE zone = 'bz2453'
AND epocutc >= '2023-07-01 00:00:00' AND epocutc <= '2023-08-31 23:59:59'
GROUP BY time;
"""
#600 is 10 minutes in seconds. 
#The idea is to convert timestamp to epoch, divide by interval desired in minutes then round to get the desired interval

crowd_data = pd.read_sql_query(query_one,con=conn)

# Timezone Conversion UTC-->CET
# cet_timezone = tz.timezone('CET')
# crowd_data['time'] = crowd_data['time'].dt.tz_localize('UTC').dt.tz_convert(cet_timezone)
# weather_data['time'] = weather_data['time'].dt.tz_localize('UTC').dt.tz_convert(cet_timezone)

print(crowd_data.columns)

# Convert 'time' to datetime and set it as the index
weather_data['time'] = pd.to_datetime(weather_data['time'])
weather_data.set_index('time', inplace=True)

# Resample the hourly data into 10-minute intervals and use the 'ffill' method to fill missing values
data_resampled = weather_data.resample('10T').first().ffill()


# Create a time range with 10-minute intervals for the entire month of August
time_range = pd.date_range(start, end, freq='10T')

# Create a DataFrame with the time range and merge it with the resampled data
result = pd.DataFrame({'time': time_range})
merged_data = pd.merge(data_resampled, crowd_data, on='time', how='outer').bfill()
merged_data = merged_data.ffill()
# Display the result
print(merged_data)
merged_data.to_csv('merged_data.csv')




fig = pt.line(data_frame=merged_data,x='time',y='people_count',width=2400,height=900)
fig.update_yaxes(rangemode='tozero',showspikes=True)
fig.update_xaxes(tickangle=45)
fig.update_layout(
    yaxis=dict(tickmode='linear',dtick=50),
    xaxis=dict(tickmode='linear',tick0='2023-07-01'),
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    )
)
fig.show()

In [ ]:
import pandas as pd
import psycopg2 as psys
import os
import sys
import plotly.express as pt
import pytz as tz
from warnings import filterwarnings
from dotenv import load_dotenv

filterwarnings("ignore", category=UserWarning, message='.*pandas only supports SQLAlchemy connectable.*')




load_dotenv() # take environment variables from .env.
%matplotlib inline

conn = None
def connect():
    try:
        print('Connecting…')
        conn = psys.connect(
            host=os.environ['DB_HOST'],
            database=os.environ['DB_NAME'],
            user=os.environ['DB_USERNAME'],
            password=os.environ['DB_PASSWORD'])
    except (Exception, psys.DatabaseError) as error:
       print(error)
       sys.exit(1)
    print('All good, Connection successful!')
    return conn

conn = connect()
cur = conn.cursor()
query_one = f"""SELECT to_timestamp(floor((extract('epoch' from epocutc)/600))*600)
AT TIME ZONE 'UTC' as time, COUNT(DISTINCT mac_address) as people_count
FROM testschemadb.flowtrack_raw_3
WHERE zone = 'bz2453'
AND epocutc >= '2023-12-01 00:00:00' AND epocutc <= '2023-12-31 23:59:59'
GROUP BY time;
"""
#600 is 10 minutes in seconds. 
#The idea is to convert timestamp to epoch, divide by interval desired in minutes then round to get the desired interval
query_two = f"""SELECT to_timestamp(floor((extract('epoch' from epocutc)/600))*600)
AT TIME ZONE 'UTC' as time, COUNT(DISTINCT mac_address) as people_count
FROM testschemadb.flowtrack_raw
WHERE zone = 'bz2453'
GROUP BY time;
"""

crowd_data = pd.read_sql_query(query_one,con=conn)
# crowd_data = pd.read_sql_query(query_two,con=conn)
# Timezone Conversion UTC-->CET
# cet_timezone = tz.timezone('CET')
# crowd_data['time'] = crowd_data['time'].dt.tz_localize('UTC').dt.tz_convert(cet_timezone)
# weather_data['time'] = weather_data['time'].dt.tz_localize('UTC').dt.tz_convert(cet_timezone)

print(crowd_data.columns)


crowd_data.to_csv('Dec.csv')




fig = pt.line(data_frame=crowd_data,x='time',y='people_count',width=2400,height=900)
fig.update_yaxes(rangemode='tozero',showspikes=True)
fig.update_xaxes(tickangle=45)
fig.update_layout(
    yaxis=dict(tickmode='linear',dtick=50),
    xaxis=dict(tickmode='linear',tick0='2023-10-01'),
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    )
)
fig.show()